In [ ]:
!pip install -r ../T2IBenchmark/models/kandinsky21/requirements.txt

In [ ]:
from T2IBenchmark import calculate_coco_fid
from T2IBenchmark.models.kandinsky21 import Kandinsky21Wrapper

**T2IModelWrapper**

To create a wrapper for your model and calculate FID-30K you simply need to inherit `T2IModelWrapper` class and implement `load_model` and `generate` methods. Example of wrapper for **Kandinsky 2.1** is listed below. You can find it in `T2IBenchmark/models/kandinsky21/wrapper.py`

In [ ]:
import torch
from PIL import Image
from T2IBenchmark import T2IModelWrapper
from kandinsky2 import get_kandinsky2


class Kandinsky21Wrapper(T2IModelWrapper):
    
    def load_model(self, device: torch.device):
        """Initialize model here"""
        self.model = get_kandinsky2(device, task_type='text2img', model_version='2.1', use_flash_attention=False)
        
    def generate(self, caption: str) -> Image.Image:
        """Generate PIL image for provided caption"""
        images = self.model.generate_text2img(
            caption, 
            num_steps=75,
            batch_size=1, 
            guidance_scale=4,
            h=768, w=768,
            sampler='p_sampler', 
            prior_cf_scale=4,
            prior_steps="5"
        )
        return images[0]

**Automatically calculate MS-COCO FID-30K**

Generations will be saved in provided dir so you can use them later

In [ ]:
fid, fid_data = calculate_coco_fid(
    Kandinsky21Wrapper,
    device='cuda:0',
    save_generations_dir='coco_generations/'
)